# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import pymongo
from pymongo import UpdateOne


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collections = db.list_collection_names()
print("Collections in 'uk_food' database:", collections)

Collections in 'uk_food' database: ['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
# Display the first document in the results using pprint
pprint(f"Number of documents: {count}")


'Number of documents: 41'


In [7]:
# Convert the result to a Pandas DataFrame
documents = list(establishments.find(query))
df = pd.DataFrame(documents)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")
      
# Display the first 10 rows of the DataFrame
pprint(df.head(10))

Number of rows in the DataFrame: 41
                        _id   FHRSID  ChangesByServerID  \
0  664e32ae1fe90211939f9348   110681                  0   
1  664e32ae1fe90211939f96c9   612039                  0   
2  664e32ae1fe90211939f99d3   730933                  0   
3  664e32ae1fe90211939f9bc1   172735                  0   
4  664e32ae1fe90211939f9bd0   172953                  0   
5  664e32ae1fe90211939fa56f   512854                  0   
6  664e32ae1fe90211939fa790  1537089                  0   
7  664e32ae1fe90211939fbcbb   155648                  0   
8  664e32ae1fe90211939fc0ff  1012883                  0   
9  664e32ae1fe90211939fc911   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532   

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": { "$regex": "London", "$options": "i" },
    "RatingValue": { "$gte": 4 }
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
      
# Display the first document in the results using pprint
pprint(f"Number of documents: {count}")


'Number of documents: 33'


In [9]:
# Convert the result to a Pandas DataFrame
documents = list(establishments.find(query))
df = pd.DataFrame(documents)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
pprint(df.head(10))

Number of rows in the DataFrame: 33
                        _id   FHRSID  ChangesByServerID  \
0  664e32ae1fe90211939fad63   621707                  0   
1  664e32ae1fe90211939fb089  1130836                  0   
2  664e32ae1fe90211939fbbd6   293783                  0   
3  664e32ae1fe90211939fc9d6  1315095                  0   
4  664e32ae1fe90211939fc9d7   294474                  0   
5  664e32ae1fe90211939fd0df   294900                  0   
6  664e32ae1fe90211939ff1ed   293756                  0   
7  664e32ae1fe90211939ff5b4   878523                  0   
8  664e32ae1fe90211939ff5ce   293772                  0   
9  664e32ae1fe90211939ff5cf   294606                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3             PI/000036464  Coombs Caterin

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [20]:


# Define the static latitude and longitude for "Penang Flavours"
latitude = 51.49014200
longitude = 0.08384000

# Create a geospatial index on the "geocode" field
index_name = establishments.create_index([("geocode", pymongo.GEOSPHERE)])

# Define the search range (0.01 degree on either side)
degree_search = 0.01

# Construct the geospatial query to find establishments within a certain distance from "Penang Flavours"
query = {
    "geocode": {
        "$nearSphere": {
            "$geometry": {
                "type": "Point",
                "coordinates": [longitude, latitude]
            },
            "$maxDistance": degree_search * 111.32 * 1000  # (1 degree is approximately 111.32 km)
        }
    },
    "RatingValue": 5
}

# Sort the establishments by hygiene score in ascending order
sort = [("scores.Hygiene", pymongo.ASCENDING)]

# Limit the results to the top 5 establishments
limit = 5

# Execute the query with a hint specifying the geospatial index
results = establishments.find(query).hint(index_name).sort(sort).limit(limit)

# Convert the MongoDB cursor to a list of dictionaries
data = list(results)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print("DataFrame:")
print(df)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
pprint(df.head(10))


DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  664e32ae1fe90211939fee1f   694635                  0   
1  664e32ae1fe90211939fee55  1392055                  0   
2  664e32ae1fe90211939fee16   695316                  0   
3  664e32ae1fe90211939fee2c  1319113                  0   
4  664e32ae1fe90211939fee21  1489067                  0   

  LocalAuthorityBusinessID              BusinessName             BusinessType  \
0             PI/000117510    Jhas Catering Services  Other catering premises   
1                    14546  Greenwich Islamic Centre   Takeaway/sandwich shop   
2             PI/000182673                McDonald's  Restaurant/Cafe/Canteen   
3                    14007          Suya Academy Ltd  Restaurant/Cafe/Canteen   
4                    15088  Wilson & Bay Coffee Club   Takeaway/sandwich shop   

   BusinessTypeID                              AddressLine1 AddressLine2  \
0            7841                        153 Plumstead Road            

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [21]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results

# Define the aggregation pipeline
pipeline = [
    {
        "$match": {
            "scores.Hygiene": 0
        }
    },
    {
        "$group": {
            "_id": "$LocalAuthorityName",
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": { 
            "count": -1 
        }
    }
]

# Execute the aggregation pipeline
results = list(establishments.aggregate(pipeline))


In [22]:
# Convert the result to a Pandas DataFrame

df = pd.DataFrame(results)

# Display the number of rows in the DataFrame

print(f"Number of rows in the DataFrame: {len(results)}")

# Display the first 10 rows of the DataFrame

print("DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 55
DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
